## Import statements for Dataframe manipulation

In [37]:
import pandas as pd
import numpy as np
import math

In [66]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/RCV Voting Method Research/Preference Profiles/ResearchProfiles/WithoutWriteins')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Create array of all files in working directory

In [67]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]

Create test array

In [50]:
files = ["Alaska22HouseDist13NoWriteins.csv"]

Create Dataframe for output

In [64]:
output = pd.DataFrame(columns = ["Election", "CondocetLoser", "MajorityLoser"])

Run eval on all files

In [68]:
for f in files:
  data = pd.read_csv(f)
  ranks = data.drop(columns = ['Num. Voters'])
  candidates = ranks.stack().unique()
  for i in range(candidates.shape[0]):
    if candidates[i] == '-':
      candidates = np.delete(candidates, i)
      break
  data = data.to_numpy()
  data[0][0]

  row = [f]
  print(f)

  #condorcet/majority evalutation
  string = condorcetLoserEval()
  row.append(string)
  string = majorityLoserEval()
  row.append(string)

  output.loc[len(output)] = row

output.to_csv("../Test.csv")

Alaska20POTUS.csv
Berkeley14CityCouncilDist1.csv
Berkeley14CityCouncilDist7.csv
Berkeley14CityCouncilDist8.csv
Berkeley17CityCouncilDist4.csv
Berkeley18CityAuditor.csv
Berkeley18CityCouncilDist1.csv
Berkeley18CityCouncilDist4.csv
Berkeley18CityCouncilDist7.csv
Berkeley18CityCouncilDist8.csv
Berkeley16CityCouncilDist2.csv
Berkeley16CityCouncilDist3.csv
Berkeley16CityCouncilDist5.csv
Berkeley16CityCouncilDist6.csv
Berkeley16Mayor.csv
Bloomington23CityCouncilDist4.csv
Bloomington23SpecialCityCouncilAtLarge.csv
Boulder23Mayor.csv
ElkRidge21Mayor.csv
Hawaii20POTUSCD1CD2.csv
Kansas20POTUS.csv
LasCruces19COUNCILORPOSITION2CITYDIST2COUNCILOR.csv
LasCruces19COUNCILORPOSITION4CITYDIST4COUNCILOR.csv
LasCruces19MAYOR.csv
Maine18DemocraticPrimaryCD2.csv
Maine20DemocraticCandidateStateHouseDistrict41.csv
Maine20DemocraticCandidateStateHouseDist47.csv
Maine20DemocraticCandidateStateHouseDist49.csv
Maine20DemocraticCandidateStateHouseDist90.csv
Maine20DemocraticCandidateStateSenateDist11.csv
Maine20Re

Condocet Loser Evaluation

In [45]:
def condorcetLoserEval():
  ##create array of points for each candidate
  points = np.full(len(candidates), 0.0)

  won = np.full(len(candidates), False)

  ##if not
  for i in range(candidates.shape[0]):
    for j in range(i + 1, candidates.shape[0]):
      a = 0
      b = 0
      for k in range(data.shape[0]):
        for l in range(data.shape[1]):
          if data[k][l] == candidates[i]:
            a = a + data[k][0]
            break
          if data[k][l] == candidates[j]:
            b = b + data[k][0]
            break
      if a < b:
        won[j] = True
      elif a > b:
        won[i] = True

  for i in range(candidates.shape[0]):
    if not won[i]:
      return candidates[i]

  return "No Candidate"

Majority Loser Evaluation

In [55]:
def majorityLoserEval():
  # If election only offers truncated ballots, there is no majority loser
  if (data.shape[1] - 1) < candidates.shape[0]:
    return "No Candidate"

  # Create array of points for each candidate
  points = np.full(len(candidates), 0.0)

  sum = 0

  # Calculate points for each candidate
  for i in range(data.shape[0]):

    if data[i][1] == '-':
      continue;

    # Calculate total points
    sum += data[i][0]

    #check if last position is empty
    if data[i][candidates.shape[0]] == '-':

      #check if second to last position is filled
      if data[i][candidates.shape[0] - 1] != '-':

        #deduce the last cand
        remainingCandidates = candidates.copy()
        for j in range(1, candidates.shape[0]):
          remainingCandidates = np.delete(remainingCandidates, np.where(remainingCandidates == data[i][j]))
        if remainingCandidates.shape[0] == 1:
          points[np.where(candidates == remainingCandidates[0])] += data[i][0]
    else:
      points[np.where(candidates == data[i][candidates.shape[0]])] += data[i][0]

  # Determine the candidate with most points
  max = 0
  maxIndex = 0
  for i in range(points.shape[0]):
    if points[i] > max:
      max = points[i]
      maxIndex = i

  # Determine if there is a majority candidate
  if (points[maxIndex] + 0.0)/(sum + 0.0) > 0.5:
    return candidates[maxIndex]
  else:
    return "No Candidate"